# 1.Load Data

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# load
dfs = pd.read_excel(r"C:\Users\tungdt174\Documents\GitHub\TungProject\Forcast and clustering\VNPTEST01.xlsx", sheet_name=None)
df_category = dfs["category"]
df_product = dfs["product"]
df_customer = dfs["customer"]
df_sale_order = dfs["sale_order"]
df_order_details = dfs["order_details"]


# parse dates
df_sale_order['order_date'] = pd.to_datetime(df_sale_order['order_date'], errors='coerce')

print("Loaded. Basic info:")
print("Sale orders:", df_sale_order.shape, "Order details:", df_order_details.shape, "Customers:", df_customer.shape)

# Quick sanity checks
print("\n-- missing values per sheet (top cols) --")
print(df_sale_order.isna().sum().loc[lambda x: x>0].head(20))
print(df_order_details.isna().sum().loc[lambda x: x>0].head(20))
print(df_customer.isna().sum().loc[lambda x: x>0].head(20))

# check duplicates
print("\n-- duplicates / unique counts --")
print("unique orders in sale_order:", df_sale_order['order_id'].nunique(), "rows:", len(df_sale_order))
print("unique orders in order_details:", df_order_details['order_id'].nunique(), "line items:", len(df_order_details))
print("unique customers:", df_customer['customer_id'].nunique())


Loaded. Basic info:
Sale orders: (20000, 3) Order details: (49815, 3) Customers: (300, 5)

-- missing values per sheet (top cols) --
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)

-- duplicates / unique counts --
unique orders in sale_order: 20000 rows: 20000
unique orders in order_details: 20000 line items: 49815
unique customers: 300


# 2. Data Cleaning

In [8]:
# strip string columns if any
for df in [df_product, df_customer, df_sale_order]:
    for c in df.select_dtypes(include='object').columns:
        df[c] = df[c].astype(str).str.strip()

# ensure numeric types
df_order_details['quantity'] = pd.to_numeric(df_order_details['quantity'], errors='coerce').fillna(0)
if 'price' in df_product.columns:
    df_product['price'] = pd.to_numeric(df_product['price'], errors='coerce')

# check negative or zero quantity/price
print("Negative quantities:", (df_order_details['quantity'] < 0).sum())
if 'price' in df_product.columns:
    print("Negative prices:", (df_product['price'] < 0).sum())


Negative quantities: 0
Negative prices: 0


In [9]:
# merge product price into order_details
df_product_price = df_product[['product_id', 'price']] if 'price' in df_product.columns else None

if df_product_price is None:
    print("Warning: product price not found. order_total will be NaN unless you add prices.")
    # Optionally set a default price or stop here.
else:
    df_merged = pd.merge(df_order_details, df_product_price, on='product_id', how='left')
    df_merged['total_price'] = df_merged['quantity'] * df_merged['price']

    # order total (one row per order_id)
    df_order_total = df_merged.groupby('order_id', as_index=False)['total_price'].sum().rename(columns={'total_price':'order_total_spend'})

    # merge with sale_order
    df_sales_full = pd.merge(df_sale_order, df_order_total, on='order_id', how='left')
    df_sales_full['order_total_spend'] = df_sales_full['order_total_spend'].fillna(0)

    # merge to customer table
    df_customer_behavior = pd.merge(df_sales_full, df_customer[['customer_id', 'name', 'gender', 'age']], on='customer_id', how='left')


# 4.Tính RFM + các chỉ số hành vi (RFM segmentation)

** RFM = Recency, Frequency, Monetary — rất hữu ích để mô tả khách hàng cốt lõi. **

In [10]:
df_customer_behavior['order_date'] = pd.to_datetime(df_customer_behavior['order_date'], errors='coerce')
analysis_date = df_customer_behavior['order_date'].max()

df_customer_summary = df_customer_behavior.groupby('customer_id').agg(
    total_spend=('order_total_spend', 'sum'),
    order_count=('order_id', 'nunique'),        # frequency = số order khác nhau
    last_order_date=('order_date', 'max'),
    first_order_date=('order_date', 'min')
).reset_index()

df_customer_summary['recency_days'] = (analysis_date - df_customer_summary['last_order_date']).dt.days
df_customer_summary['tenure_days'] = (df_customer_summary['last_order_date'] - df_customer_summary['first_order_date']).dt.days

# average order value per customer
df_customer_summary['average_order_value'] = df_customer_summary['total_spend'] / df_customer_summary['order_count']

# merge back demographic fields if needed
df_customer_summary = df_customer_summary.merge(df_customer[['customer_id','name','gender','age']], on='customer_id', how='left')
